In [10]:
import gridworld

# Define parameters for the gridworld
grid_size = [10.0, 10.0]
step_size = 0.5
goal = [8.0, 8.0]
obstacles = [
    [[2.0, 2.0], [4.0, 4.0]],  # Example obstacle as a parallelogram
    [[6.0, 6.0], [7.0, 7.0]]
]
stretch = [1.0, 1.0]  # Default stretch factors
shear = [0.0, 0.0]  # Default shear factors
observation_radius = 1.0
shear_range = [-0.5, 0.5]
stretch_range = [0.8, 1.2]

# Instantiate the ObservableDeformedGridworld
env = gridworld.ObservableDeformedGridworld(
    grid_size,
    step_size,
    goal,
    obstacles,
    stretch,
    shear,
    observation_radius,
    shear_range,
    stretch_range
)

# Reset the environment
env.reset(42)

# Example interaction loop
for _ in range(10):
    action = 0  # Example action (0: move up)
    observation, reward, terminated,truncated, info = env.step(action)
    print(f"Observation: {observation}, Reward: {reward}, Terminated: {terminated}, Info: {info}")
    if terminated:
        break


Observation: {'theta': [1.1186171937151383, 0.27969099761266125, 0.09685016158005655, 0.8733739151573474], 'pos': [0.0, 0.0]}, Reward: -2.0, Terminated: False, Info: {'collision': True}
Observation: {'theta': [1.1186171937151383, 0.27969099761266125, 0.09685016158005655, 0.8733739151573474], 'pos': [0.0, 0.0]}, Reward: -2.0, Terminated: False, Info: {'collision': True}
Observation: {'theta': [1.1186171937151383, 0.27969099761266125, 0.09685016158005655, 0.8733739151573474], 'pos': [0.0, 0.0]}, Reward: -2.0, Terminated: False, Info: {'collision': True}
Observation: {'theta': [1.1186171937151383, 0.27969099761266125, 0.09685016158005655, 0.8733739151573474], 'pos': [0.0, 0.0]}, Reward: -2.0, Terminated: False, Info: {'collision': True}
Observation: {'theta': [1.1186171937151383, 0.27969099761266125, 0.09685016158005655, 0.8733739151573474], 'pos': [0.0, 0.0]}, Reward: -2.0, Terminated: False, Info: {'collision': True}
Observation: {'theta': [1.1186171937151383, 0.27969099761266125, 0.096

In [15]:
class Grid(gridworld.ObservableDeformedGridworld):
    def __init__(self, grid_size, step_size, goal, obstacles, stretch, shear, observation_radius, shear_range, stretch_range):
        super().__init__(grid_size, step_size, goal, obstacles, stretch, shear, observation_radius, shear_range, stretch_range)

env = Grid(grid_size, step_size, goal, obstacles, stretch, shear, observation_radius, shear_range, stretch_range)